In [ ]:
# please download the input data file here

# https://drive.google.com/file/d/1qWrN8qvDpv1MBQDpSPRHPmAuAAFo7Hyb/view?usp=sharing

# Input Sentence + Op term 1

# Output= Categort 1, polarity 1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
df=pd.read_excel('/content/UC_Dataset.xlsx')
df.columns = df.columns.str.replace(' ', '')

In [5]:
df.columns

Index(['Sentences', 'OpinionTerm1', 'AspectCategory1', 'AspectPolarity1',
       'OpinionTerm2', 'AspectCategory2', 'Aspectpolarity2'],
      dtype='object')

In [3]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [6]:
# df['Aspect_Category1'].value_counts()
selected_columns = ['Sentences', 'OpinionTerm1','AspectCategory1','AspectPolarity1', "OpinionTerm2", "AspectCategory2", "Aspectpolarity2"]
df_data = df[selected_columns]

In [7]:
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
nltk.download('punkt')
nltk.download('stopwords')
# Preprocessing
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)


def preprocess_text(text):
    text = str(text)
    tokens = nltk.word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stopwords and token not in punctuation]
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**AspectCategory**

In [9]:
df_data['OpinionTerm2'].fillna(method='ffill', inplace=True)  # Forward fill
df_data['OpinionTerm2'].fillna(method='bfill', inplace=True)
df_data['AspectCategory2'].fillna(method='ffill', inplace=True)  # Forward fill
df_data['AspectCategory2'].fillna(method='bfill', inplace=True)
df_data['Aspectpolarity2'].fillna(method='ffill', inplace=True)  # Forward fill
df_data['Aspectpolarity2'].fillna(method='bfill', inplace=True)

In [10]:
df_data = df_data.copy()
df_data['Sentences' ] = df_data['Sentences'].apply(preprocess_text)
df_data['OpinionTerm1' ] = df_data[ 'OpinionTerm1'].apply(preprocess_text)
# df_data['opinion_term' ] = df_data[ 'opinion_term'].apply(preprocess_text)

In [11]:
# Xfeatures = df_data[['Sentences', 'Opinion_Term', 'Opinion_Term']]
Xfeatures = df_data[['Sentences', 'OpinionTerm1']]
ylabels = df_data[['AspectCategory1', 'AspectPolarity1']]
# ylabels = df_data[['Aspect_Category1', 'aspect_category2', 'Aspect_Polarity1', 'Aspect_polarity2']]

In [26]:
# Random Forest clasifier
Xfeatures["combined"] = Xfeatures["Sentences"]+ " " + Xfeatures['OpinionTerm1']

X = Xfeatures["combined"]
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

<ipython-input-26-f94a4c94cb36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xfeatures["combined"] = Xfeatures["Sentences"]+ " " + Xfeatures['OpinionTerm1']


In [19]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier


classifiers = [DecisionTreeClassifier, RandomForestClassifier, ExtraTreeClassifier,
               ExtraTreesClassifier, KNeighborsClassifier, RadiusNeighborsClassifier]
dict_classifiers = {
    "SVM":svm.SVC(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "Naive B":  MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),

}
data = df = pd.DataFrame(columns = ['model', 'Accuracy_AC1',  'Accuracy_AP1'])
for model, model_instantiation in dict_classifiers.items():
  clf = MultiOutputClassifier(model_instantiation)
  clf.fit(X_train_vec, y_train)
  y_pred = clf.predict(X_test_vec)

  AC_1 = accuracy_score(y_test["AspectCategory1"].values, y_pred[:,0])
  AP_1 = accuracy_score(y_test["AspectPolarity1"].values, y_pred[:,1])
  data.loc[len(data)] = [model, AC_1, AP_1]
  print(classification_report(y_test["AspectCategory1"].values, y_pred[:,0], zero_division=0))
  print(classification_report(y_test["AspectPolarity1"].values, y_pred[:,1], zero_division=0))
  report1 = classification_report(y_test["AspectCategory1"].values, y_pred[:,0], zero_division=1, output_dict=True)
  report2 = classification_report(y_test["AspectPolarity1"].values, y_pred[:,1], zero_division=1, output_dict=True)
  df_report1 = pd.DataFrame(report1).transpose()
  df_report2 = pd.DataFrame(report2).transpose()

                                                                      precision    recall  f1-score   support

                                                   App_Functionality       1.00      0.31      0.47        13
                                                              App_UI       0.83      0.59      0.69        17
                                                         App_Utility       0.88      0.90      0.89       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.73      0.95      0.83       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.53      0.57      0.55        14
         

In [20]:
print( df_report1)

                                                    precision    recall  \
App_Functionality                                    0.666667  0.307692   
App_UI                                               0.583333  0.411765   
App_UI                                               0.000000  1.000000   
App_Utility                                          0.869231  0.889764   
App_Utility                                          1.000000  0.000000   
Company_Service                                      0.708042  0.884279   
Company_Service                                      1.000000  0.000000   
Company_service                                      0.000000  0.000000   
Cost                                                 0.555556  0.357143   
Customer_Support                                     0.642857  0.750000   
Payment                                              1.000000  0.000000   
cost                                                 0.750000  0.272727   
payment                  

In [21]:
print( df_report2)

              precision    recall  f1-score     support
Positive       0.513889  0.579937  0.544919  319.000000
negative       0.821429  0.766667  0.793103  180.000000
positive       0.483444  0.450617  0.466454  324.000000
positive       0.000000  0.000000  0.000000   10.000000
accuracy       0.563025  0.563025  0.563025    0.563025
macro avg      0.454690  0.449305  0.451119  833.000000
weighted avg   0.562333  0.563025  0.561487  833.000000


In [24]:
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn import svm
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.neighbors import KNeighborsClassifier

# dict_classifiers = {
#     "SVM":svm.SVC(),
#     "KNN":KNeighborsClassifier(n_neighbors=3),
#     "Naive B":  MultinomialNB(force_alpha=True),
#     "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),

# }
# data = df = pd.DataFrame(columns = ['model', 'Accuracy_AC1'])
# for model, model_instantiation in dict_classifiers.items():
#   clf = MultiOutputClassifier(model_instantiation)
#   clf.fit(X_train_vec, y_train)
#   y_pred = clf.predict(X_test_vec)
#   AC_1 = accuracy_score(y_test["AspectCategory1"].values, y_pred)
#   data.loc[len(data)] = [model, AC_1]
#   conf_matrix = confusion_matrix(y_test["AspectCategory1"].values, y_pred)
#   print(classification_report(y_test["AspectCategory1"].values, y_pred, zero_division=0))


ValueError: ignored

In [28]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

# Assuming X_train_vec, y_train, X_test_vec, y_test are defined

dict_classifiers = {
    "SVM": svm.SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Naive B": MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
}

data = pd.DataFrame(columns=['model', 'Accuracy_AC1'])

for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)

    # Extract predictions for AspectCategory1
    y_pred_ac1 = y_pred[:, 0]  # Assuming AspectCategory1 is the first output

    AC_1 = accuracy_score(y_test["AspectCategory1"].values, y_pred_ac1)
    data = data.append({'model': model, 'Accuracy_AC1': AC_1}, ignore_index=True)

    conf_matrix = confusion_matrix(y_test["AspectCategory1"].values, y_pred_ac1)
    print(classification_report(y_test["AspectCategory1"].values, y_pred_ac1, zero_division=0))

# Now data DataFrame contains the models and their corresponding accuracies for AspectCategory1


<ipython-input-28-5090a6729eb6>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Accuracy_AC1': AC_1}, ignore_index=True)


                                                                      precision    recall  f1-score   support

                                                   App_Functionality       1.00      0.31      0.47        13
                                                              App_UI       0.83      0.59      0.69        17
                                                         App_Utility       0.88      0.90      0.89       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.73      0.95      0.83       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.53      0.57      0.55        14
         

<ipython-input-28-5090a6729eb6>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Accuracy_AC1': AC_1}, ignore_index=True)
<ipython-input-28-5090a6729eb6>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Accuracy_AC1': AC_1}, ignore_index=True)


                                                                      precision    recall  f1-score   support

                                                   App_Functionality       0.27      0.23      0.25        13
                                                              App_UI       0.77      0.59      0.67        17
                                                         App_Utility       0.82      0.83      0.82       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.74      0.85      0.79       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.58      0.50      0.54        14
         

<ipython-input-28-5090a6729eb6>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Accuracy_AC1': AC_1}, ignore_index=True)


In [29]:
data

,model,Accuracy_AC1
0,SVM,0.749100
1,KNN,0.701080
2,Naive B,0.650660
3,XGBoost,0.687875


In [ ]:
# # This part is extra -- Just if you want to plot confuson matrix
# %matplotlib inline
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(7.5, 7.5))
# ax.matshow(conf_matrix, cmap="Greens", alpha=0.3)
# for i in range(conf_matrix.shape[0]):
#     for j in range(conf_matrix.shape[1]):
#         ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')

# plt.xlabel('Predictions', fontsize=18)
# plt.ylabel('Actuals', fontsize=18)
# plt.title('Confusion Matrix', fontsize=18)
# plt.show()

In [23]:
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.metrics import precision_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn import svm
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.neighbors import KNeighborsClassifier

# dict_classifiers = {
#     "SVM": svm.SVC(),
#     "KNN": KNeighborsClassifier(n_neighbors=3),
#     "Naive B": MultinomialNB(force_alpha=True),
#     "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
# }
# data = pd.DataFrame(columns=['model', 'Precision_AC1'])
# for model, model_instantiation in dict_classifiers.items():
#     clf = MultiOutputClassifier(model_instantiation)
#     clf.fit(X_train_vec, y_train)
#     y_pred = clf.predict(X_test_vec)
#     precision_AC1 = precision_score(y_test['AspectCategory1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
#     data.loc[len(data)] = [model, precision_AC1]

ValueError: ignored

In [30]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, classification_report
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

# Assuming X_train_vec, y_train, X_test_vec, y_test are defined

dict_classifiers = {
    "SVM": svm.SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Naive B": MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
}

data = pd.DataFrame(columns=['model', 'Precision_AC1'])

for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)

    # Extract predictions for AspectCategory1
    y_pred_ac1 = y_pred[:, 0]  # Assuming AspectCategory1 is the first output

    precision_AC1 = precision_score(y_test['AspectCategory1'].values, y_pred_ac1, average='macro')  # Change 'macro' to another option if needed
    data = data.append({'model': model, 'Precision_AC1': precision_AC1}, ignore_index=True)

    # Additional metrics if needed
    print(classification_report(y_test['AspectCategory1'].values, y_pred_ac1, zero_division=0))

# Now data DataFrame contains the models and their corresponding precision for AspectCategory1


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-30-7b085069be76>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Precision_AC1': precision_AC1}, ignore_index=True)


                                                                      precision    recall  f1-score   support

                                                   App_Functionality       1.00      0.31      0.47        13
                                                              App_UI       0.83      0.59      0.69        17
                                                         App_Utility       0.88      0.90      0.89       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.73      0.95      0.83       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.53      0.57      0.55        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-30-7b085069be76>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Precision_AC1': precision_AC1}, ignore_index=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-30-7b085069be76>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. U

                                                                      precision    recall  f1-score   support

                                                   App_Functionality       0.27      0.23      0.25        13
                                                              App_UI       0.77      0.59      0.67        17
                                                         App_Utility       0.82      0.83      0.82       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.74      0.85      0.79       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.58      0.50      0.54        14
         

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
<ipython-input-30-7b085069be76>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'Precision_AC1': precision_AC1}, ignore_index=True)


In [31]:
data

,model,Precision_AC1
0,SVM,0.369932
1,KNN,0.306901
2,Naive B,0.154322
3,XGBoost,0.276058


In [33]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, classification_report
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

# Assuming X_train_vec, y_train, X_test_vec, y_test are defined

dict_classifiers = {
    "SVM": svm.SVC(),
    "KNN": KNeighborsClassifier(n_neighbors=3),
    "Naive B": MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0),
}

data = pd.DataFrame(columns=['model', 'F1_Score_AC1'])

for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)

    # Extract predictions for AspectCategory1
    y_pred_ac1 = y_pred[:, 0]  # Assuming AspectCategory1 is the first output

    f1_score_AC1 = f1_score(y_test['AspectCategory1'].values, y_pred_ac1, average='macro')  # Change 'macro' to another option if needed
    data = data.append({'model': model, 'F1_Score_AC1': f1_score_AC1}, ignore_index=True)

    # Additional metrics if needed
    print(classification_report(y_test['AspectCategory1'].values, y_pred_ac1, zero_division=0))

# Now data DataFrame contains the models and their corresponding F1 scores for AspectCategory1


<ipython-input-33-af0982e809db>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'F1_Score_AC1': f1_score_AC1}, ignore_index=True)


                                                                      precision    recall  f1-score   support

                                                   App_Functionality       1.00      0.31      0.47        13
                                                              App_UI       0.83      0.59      0.69        17
                                                         App_Utility       0.88      0.90      0.89       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.73      0.95      0.83       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.53      0.57      0.55        14
         

<ipython-input-33-af0982e809db>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'F1_Score_AC1': f1_score_AC1}, ignore_index=True)
<ipython-input-33-af0982e809db>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'F1_Score_AC1': f1_score_AC1}, ignore_index=True)


                                                                      precision    recall  f1-score   support

                                                   App_Functionality       0.27      0.23      0.25        13
                                                              App_UI       0.77      0.59      0.67        17
                                                         App_Utility       0.82      0.83      0.82       127
                                                        App_Utility        0.00      0.00      0.00         3
                                                     Company_Service       0.74      0.85      0.79       458
                                                    Company_Service        0.00      0.00      0.00         2
                                                     Company_service       0.00      0.00      0.00         2
                                                                Cost       0.58      0.50      0.54        14
         

<ipython-input-33-af0982e809db>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({'model': model, 'F1_Score_AC1': f1_score_AC1}, ignore_index=True)


In [34]:
data

,model,F1_Score_AC1
0,SVM,0.282069
1,KNN,0.291495
2,Naive B,0.090216
3,XGBoost,0.231925


In [37]:
# Xfeatures = df_data[['Sentences', 'Opinion_Term', 'Opinion_Term']]
Xfeatures = df_data[['OpinionTerm1','AspectCategory1']]
ylabels = df_data[['AspectPolarity1']]
# ylabels = df_data[['Aspect_Category1', 'aspect_category2', 'Aspect_Polarity1', 'Aspect_polarity2']]

**SentimentPolairty**

In [38]:
ylabels = df_data[['AspectPolarity1']]
Xfeatures["combined"] =  Xfeatures['OpinionTerm1']+ " " + Xfeatures['AspectCategory1']
X = Xfeatures["combined"]
X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

# Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

<ipython-input-38-47a93e848d8c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Xfeatures["combined"] =  Xfeatures['OpinionTerm1']+ " " + Xfeatures['AspectCategory1']


In [39]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

dict_classifiers = {
    "SVM":svm.SVC(),
    "KNN":KNeighborsClassifier(n_neighbors=3),
    "Naive B":  MultinomialNB(force_alpha=True),
    "XGBoost": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0),

}
data = df = pd.DataFrame(columns = ['model', 'Accuracy_AP1'])
for model, model_instantiation in dict_classifiers.items():
  clf = MultiOutputClassifier(model_instantiation)
  clf.fit(X_train_vec, y_train)
  y_pred = clf.predict(X_test_vec)
  AP_1 = accuracy_score(y_test["AspectPolarity1"].values, y_pred)
  data.loc[len(data)] = [model, AP_1]
  conf_matrix = confusion_matrix(y_test["AspectPolarity1"].values, y_pred)
  print(classification_report(y_test["AspectPolarity1"].values, y_pred, zero_division=0))

              precision    recall  f1-score   support

    Positive       0.56      0.75      0.64       319
   negative        0.84      0.82      0.83       180
    positive       0.59      0.41      0.49       324
   positive        0.00      0.00      0.00        10

    accuracy                           0.62       833
   macro avg       0.50      0.49      0.49       833
weighted avg       0.62      0.62      0.61       833

              precision    recall  f1-score   support

    Positive       0.51      0.61      0.56       319
   negative        0.83      0.76      0.79       180
    positive       0.53      0.47      0.50       324
   positive        0.00      0.00      0.00        10

    accuracy                           0.58       833
   macro avg       0.47      0.46      0.46       833
weighted avg       0.58      0.58      0.58       833

              precision    recall  f1-score   support

    Positive       0.55      0.71      0.62       319
   negative        0.

In [40]:
data

,model,Accuracy_AP1
0,SVM,0.624250
1,KNN,0.581032
2,Naive B,0.612245
3,XGBoost,0.602641


In [41]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
data = pd.DataFrame(columns=['model', 'Precision_AP1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    precision_AC1 = precision_score(y_test['AspectPolarity1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, precision_AC1]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
data

,model,Precision_AP1
0,SVM,0.495014
1,KNN,0.467145
2,Naive B,0.486713
3,XGBoost,0.477429


In [43]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn import svm
data = pd.DataFrame(columns=['model', 'Recall_AP1'])
for model, model_instantiation in dict_classifiers.items():
    clf = MultiOutputClassifier(model_instantiation)
    clf.fit(X_train_vec, y_train)
    y_pred = clf.predict(X_test_vec)
    precision_AC1 = precision_score(y_test['AspectPolarity1'].values, y_pred, average='macro')  # Change 'macro' to another option if needed
    data.loc[len(data)] = [model, precision_AC1]

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [44]:
data

,model,Recall_AP1
0,SVM,0.495014
1,KNN,0.467145
2,Naive B,0.486713
3,XGBoost,0.477429
